# LangChain: The LLM Application Framework

LangChain, an open-source library, empowers developers by providing a standardized and structured interface for building and integrating various components of an LLM Application. Its model-agnostic nature allows for compatibility with models from multiple LLM providers, including OpenAI, HuggingFace, and others. 

Using Langchain allows us to build ("like a chain") reusable components as part of complex multi-step LLM-based applications clearly and succinctly. 

You can learn about different [LangChain components here.](https://python.langchain.com/v0.2/docs/concepts/#components)

This tutorial will focus on a few LangChain components and learn about `chaining,` one of its powerful features.

## [Prompt templates](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates)

Prompt Templates provides templates for designing prompts fed as inputs to the LLM models.
It helps us design templates with multiple inputs that are parameterized and reusable.

Below is an example of how to use a prompt template.

In [ ]:
import langchain

In [ ]:
from langchain import PromptTemplate
from llama_cpp import Llama
from ssec_tutorials import OLMO_MODEL
from ssec_tutorials.scipy_conf import *  # Contains helper methods for tutorial

### [String PromptTemplates](https://python.langchain.com/v0.2/docs/concepts/#string-prompttemplates)

Prompt templates to format a single string

In [ ]:
prompt_template = PromptTemplate.from_template(
    "{planet_name} in the solar system is the "
)

prompt_template.format(planet_name="Mars")

In [ ]:
olmo = Llama(model_path=str(OLMO_MODEL), verbose=False)

In [ ]:
model_response = olmo(
    prompt=prompt_template.format(planet_name="Mars"),
    temperature=0.2,
    max_tokens=8,
    echo=True,
)  # Generate a completion, can also call olmo.create_completion

In [ ]:
print(parse_text_generation_response(model_response))

In [ ]:
# Another example
prompt_template = PromptTemplate.from_template(
    "{entity_1} of the planet {entity_2} is "
)
prompt_template.format(entity_1="Size", entity_2="Earth")

In [ ]:
model_response = olmo(
    prompt=prompt_template.format(entity_1="Size", entity_2="Earth"),
    temperature=0.2,
    echo=True,
)

In [ ]:
print(parse_text_generation_response(model_response))

## LLM Interface

LangChain provides us with a standardized interface for loading the LLM model. Once standardized, we can use the same methods across models from different providers to call/invoke functions, enabling reusability.

Loading the model via [LangChain's LlamaCpp](https://python.langchain.com/v0.1/docs/integrations/llms/llamacpp/) abstraction enables us to use the `chaining` feature.

In [ ]:
from langchain_community.llms import LlamaCpp

In [ ]:
olmo = LlamaCpp(
    model_path=str(OLMO_MODEL),
    temperature=0.8,
    # stop=["."],
    verbose=False,
)

In [ ]:
# Create a prompt template using OLMo's tokenizer chat template we saw in module 1.
prompt_template = PromptTemplate.from_template(
    template=olmo.client.metadata["tokenizer.chat_template"], template_format="jinja2"
)

In [ ]:
# This is how you can format the prompt message
prompt_template.format(
    messages=[
        {
            "role": "user",
            "content": "You are a helpful assistant. Tell me a joke about cats",
        }
    ],
    add_generation_prompt=True,
    eos_token="<|endoftext|>",
)

## Chain in LangChain

Chaining allows us to combine multiple components, as described above, in series or parallel to develop a multi-step LLM pipeline.
As shown in the image below, any number of components can be linked together to form a chain.

![LancChain Chain](../../images/langchain-chain.webp)


Image Source: [www.analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2023/10/a-comprehensive-guide-to-using-chains-in-langchain/)

Internally, the chain works like below:

STEP 1: Dictionary is processed as an input to the prompt template.  
STEP 2: Prompt Template reads the variables to form the prompt text as output - "What are stars and moon?"  
STEP 3: The prompt is given as input to the LLM model.  
STEP 4: LLM Model produces output.  
STEP 5: The output goes through StrOutputParser that parses it into string and gives the result.  

We can use the pipe operator ("|"), which is part of the [LCEL(Lang Chain Expression Language)](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel). The pipe operator sequentially arranges each component, similar to the above image.

In [ ]:
llm_chain = prompt_template | olmo

In [ ]:
# Construct the prompt as expected by OLMo
llm_chain.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "You are a helpful assistant. Tell me a joke about cats",
            }
        ],
        "add_generation_prompt": True,
        "eos_token": "<|endoftext|>",
    }
)

Instead of having to invoke `llm_chain` repeatedly with `add_generation_prompt` and `eos_token`, we can update our `prompt_template`.

In [ ]:
# Create a prompt template using OLMo's tokenizer chat template we saw in module 1, but this time use partial variables.
prompt_template = PromptTemplate.from_template(
    template=olmo.client.metadata["tokenizer.chat_template"],
    template_format="jinja2",
    partial_variables={"add_generation_prompt": True, "eos_token": "<|endoftext|>"},
)

In [ ]:
llm_chain = prompt_template | olmo

Let's stream the output instead of waiting for OLMo to generate and display the text. We can use [Callbacks](https://python.langchain.com/v0.2/docs/concepts/#callbacks) to subscribe to various events in your LLM application pipeline. Check [this out](https://python.langchain.com/v0.1/docs/modules/callbacks/#callback-handlers) for a list of events. 

In [ ]:
from langchain_core.callbacks import StreamingStdOutCallbackHandler

In [ ]:
llm_chain.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "You are a helpful assistant. Tell me a joke about cats",
            }
        ]
    },
    config={"callbacks": [StreamingStdOutCallbackHandler()]},
)

We will cover more LangChain concepts in upcoming notebooks. 